In [1]:
# !pip3 install sacrebleu

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
from sacrebleu.metrics import BLEU, CHRF, TER

bleu = BLEU()
chrf = CHRF(word_order = 2)

In [4]:
from transformers import T5Config, TFT5Model, T5Model, load_tf_weights_in_t5, T5Tokenizer
from transformers import T5ForConditionalGeneration, TFT5ForConditionalGeneration

In [5]:
model_tf = TFT5ForConditionalGeneration.from_pretrained('mesolitica/t5-small-finetuned-noisy-en-ms')

2022-08-01 01:38:36.798298: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-08-01 01:38:36.798336: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: husein-MS-7D31
2022-08-01 01:38:36.798340: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: husein-MS-7D31
2022-08-01 01:38:36.798533: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.129.6
2022-08-01 01:38:36.798549: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.129.6
2022-08-01 01:38:36.798552: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 470.129.6
2022-08-01 01:38:36.798883: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU i

In [6]:
tokenizer = T5Tokenizer.from_pretrained('mesolitica/t5-small-finetuned-noisy-en-ms')

In [7]:
from unidecode import unidecode

with open('eng_Latn.dev') as fopen:
    eng = fopen.read().split('\n')
    
with open('zsm_Latn.dev') as fopen:
    ms = fopen.read().split('\n')
    
right = [unidecode(s) for s in ms]
left = [unidecode(s) for s in eng]

In [8]:
len(ms), len(eng)

(998, 998)

In [9]:
from tqdm import tqdm

batch_size = 16

results = []
for i in tqdm(range(0, len(left), batch_size)):
    input_ids = [{'input_ids': tokenizer.encode(f'terjemah Inggeris ke Melayu: {s}', return_tensors = 'tf')[0]} for s in left[i:i + batch_size]]
    padded = tokenizer.pad(input_ids, padding = 'longest')
    outputs = model_tf.generate(**padded, max_length = 1000)
    for o in outputs:
        results.append(tokenizer.decode(o, skip_special_tokens=True))

  0%|                                                                                                         | 0/63 [00:00<?, ?it/s]2022-08-01 01:38:44.700456: I tensorflow/compiler/xla/service/service.cc:171] XLA service 0xe924dc0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-08-01 01:38:44.700478: I tensorflow/compiler/xla/service/service.cc:179]   StreamExecutor device (0): Host, Default Version
2022-08-01 01:38:44.711646: I tensorflow/compiler/jit/xla_compilation_cache.cc:363] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
100%|████████████████████████████████████████████████████████████████████████████████████████████████| 63/63 [02:59<00:00,  2.85s/it]


In [10]:
len(results)

998

In [11]:
filtered_left, filtered_right = [], []
for no, r in enumerate(results):
    if len(r):
        filtered_left.append(r)
        filtered_right.append(right[no])

In [12]:
refs = [filtered_right]
sys = filtered_left

In [13]:
r = bleu.corpus_score(sys, refs)

In [14]:
r.__dict__

{'name': 'BLEU',
 'score': 42.11032800535816,
 '_mean': -1.0,
 '_ci': -1.0,
 '_verbose': '73.2/50.3/35.8/25.8 (BP = 0.980 ratio = 0.980 hyp_len = 21597 ref_len = 22027)',
 'bp': 0.9802867258527527,
 'counts': [15815, 10355, 7025, 4803],
 'totals': [21597, 20600, 19603, 18606],
 'sys_len': 21597,
 'ref_len': 22027,
 'precisions': [73.22776311524748,
  50.26699029126213,
  35.836351578839974,
  25.81425346662367],
 'prec_str': '73.2/50.3/35.8/25.8',
 'ratio': 0.9804785036546058}

In [15]:
chrf.corpus_score(sys, refs)

chrF2++ = 66.31